# Consumer & Business Confidence

---

A look at the consumer and business confidence indexes, according to the OECD.

In [1]:
import pandas as pd
import altair as alt

from urllib.request import urlopen, Request
from io import BytesIO

In [2]:
baseUrl = "https://stats.oecd.org/sdmx-json/data/DP_LIVE/{}/OECD?contentType=csv&detail=code&separator=comma&csv-lang=en"

uaString = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'

def fetchSentiment(url, retries=5):
    if retries < 0:
        return None
    try:
        req = Request(url, data=None, headers={ 'User-Agent': uaString })

        data = urlopen(req)
        
        return pd.read_csv(BytesIO(data.read()))
    except:
        return fetch(url, retries-1)

df_consumer = fetchSentiment(baseUrl.format("USA.CCI..."))

#df_consumer.head()

df_business = fetchSentiment(baseUrl.format(".BCI..."))

#df_business.head()

In [3]:
cUsaSubset = df_consumer[df_consumer['LOCATION'] == 'USA'][['TIME', 'Value']]

def chartFor(df, label, color='green'):
    return alt.Chart(df).mark_line(color=f'{color}').transform_calculate(
        label=f"'{label}'"
    ).encode(
        alt.X('TIME:T', axis=alt.Axis(title='')),
        alt.Y('Value:Q', scale=alt.Scale(zero=False)),
        alt.Opacity('label:N', legend=alt.Legend(title=""))
    ).properties(
        width=750,
        height=400
    )

consumer = chartFor(cUsaSubset, 'Consumer')

consumer.display()

alt.Chart(...)

In [4]:
bUsaSubset = df_business[df_business['LOCATION'] == 'USA'][['TIME', 'Value']]

business = chartFor(bUsaSubset, 'Business', 'blue')

business.display()

alt.Chart(...)

In [5]:
consumer + business

alt.LayerChart(...)

In [6]:
cUsaYoy = cUsaSubset.set_index('TIME').pct_change(12).apply(lambda v: v * 100).reset_index()
bUsaYoy = bUsaSubset.set_index('TIME').pct_change(12).apply(lambda v: v * 100).reset_index()

def yoyChartFor(df, label, color='navy'):
    return alt.Chart(df.dropna()[-180:]).mark_bar(width=1.5, color=f'{color}').transform_calculate(
        label=f"'{label}'",
        change_in_percent="format(datum.Value, ',.2f')",
    ).encode(
        alt.X('TIME:T', axis=alt.Axis(title='')),
        alt.Y('Value:Q', axis=alt.Axis(title='Year-over-Year Change [%]')),
        alt.Opacity('label:O', legend=alt.Legend(title="")),
        tooltip=[alt.Tooltip('TIME:T', format='%b %Y'), alt.Tooltip('change_in_percent:Q')]
    ).properties(
        title=f"Changing {label} Sentiment [OECD]",
        width=750,
        height=400
    )

cYoy = yoyChartFor(cUsaYoy, 'Consumer', 'green')
cYoy.display()

alt.Chart(...)

In [7]:
bYoy = yoyChartFor(bUsaYoy, 'Business', 'black')
bYoy.display()

alt.Chart(...)

In [8]:
c = (cYoy + bYoy).properties(title='Changing US Sentiment [OECD]', background='white')

c.save('sentiment.png')
c.display()

alt.LayerChart(...)